# Load Data

run save data cells in projet.ipynb before

In [1]:
import pandas as pd

movies = pd.read_csv('../data/save/movies.csv')
ratings = pd.read_csv('../data/save/ratings.csv')
users = pd.read_csv('../data/save/users.csv')

In [2]:
display(movies.head())
display(ratings.head())
display(users.head())

,MovieId,Title,Year,isAdult,runtimeMinutes,averageRating,numVotes,Action,Adult,Adventure,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1,toy story,1995,0,81,4.15,1073870,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,jumanji,1995,0,104,3.55,379231,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,grumpier old men,1995,0,101,3.30,29839,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,waiting to exhale,1995,0,124,3.00,12277,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5,father of the bride part ii,1995,0,106,3.05,41877,0,0,0,...,0,0,0,0,1,0,0,0,0,0


,UserId,MovieId,Rating
0,1,1193,5.0
1,1,661,3.0
2,1,914,3.0
3,1,3408,4.0
4,1,2355,5.0


,UserId,Gender,Age
0,1,1,1
1,2,0,56
2,3,0,25
3,4,0,45
4,5,0,25


# NNE --

Le modèle NNE est l'un des essais. La raison de son abandon est ses résultats non satisfaisant

In [48]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Flatten, Concatenate, Input
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [55]:
full_data = pd.merge(pd.merge(ratings, users, on='UserId'), movies, on='MovieId')

# Sélectionner les colonnes pour item_train et user_train
item_features = movies.columns.difference(['Title', 'Year', 'runtimeMinutes', 'MovieId']).tolist()
item_train = full_data[item_features].values
user_train = full_data[['Gender', 'Age']].values
y_train = full_data['Rating'].values

# Définir les dimensions des entrées utilisateur et item
num_user_features = user_train.shape[1]
num_item_features = item_train.shape[1]

# Sauvegarder les données non scalées
item_train_unscaled = item_train.copy()
user_train_unscaled = user_train.copy()
y_train_unscaled = y_train.copy()

In [56]:
# Appliquer le scaling
scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))

# Inverser les transformations pour vérification
inverse_item = scalerItem.inverse_transform(item_train)
inverse_user = scalerUser.inverse_transform(user_train)

# Vérifier que les scalers peuvent inverser correctement les transformations
assert np.allclose(item_train_unscaled, inverse_item), "Something is wrong with the item scaler"
assert np.allclose(user_train_unscaled, inverse_user), "Something is wrong with the user scaler"

print("\033[92m" + "All tests passed!" + "\033[0m")

All tests passed!


In [57]:
# Split le dataset
user_item_data = np.concatenate([user_train, item_train], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    user_item_data, y_train, train_size=0.80, shuffle=True, random_state=1
)

user_train = X_train[:, :num_user_features]
item_train = X_train[:, num_user_features:]
user_test = X_test[:, :num_user_features]
item_test = X_test[:, num_user_features:]

print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (727000, 27)
movie/item test data shape: (181750, 27)


In [58]:
# Définir le modèle
num_outputs = 32
tf.random.set_seed(1)

user_NN = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(num_outputs),
    ]
)

item_NN = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(num_outputs),
    ]
)

# creer le input des utilisateurs et calculer la norme L2
input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vu)

# Creer le input des films et calculer la norme L2
input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm)

# Calculer le produit scalaire
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# Créer le model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_10      │ (None, 27)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_4        │ (None, 32)        │     37,792 │ input_layer_8[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_5        │ (None, 32)        │     44,192 │ input_layer_10[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None, 32)        │          0 │ sequential_4[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_5 (Lambda)   │ (None, 32)        │          0 │ sequential_5[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_2 (Dot)         │ (None, 1)         │          0 │ lambda_4[0][0],   │
│                     │                   │            │ lambda_5[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 81,984 (320.25 KB)

 Trainable params: 81,984 (320.25 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraîner le modèle
history = model.fit([user_train, item_train], y_train, epochs=6, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss = model.evaluate([user_test, item_test], y_test)
print(f'Test Loss: {loss}')


All tests passed!
movie/item training data shape: (727000, 27)
movie/item test data shape: (181750, 27)


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 27)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, 32)        │     37,792 │ input_layer_4[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_3        │ (None, 32)        │     44,192 │ input_layer_6[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 32)        │          0 │ sequential_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 32)        │          0 │ sequential_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, 1)         │          0 │ lambda_2[0][0],   │
│                     │                   │            │ lambda_3[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 81,984 (320.25 KB)

 Trainable params: 81,984 (320.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6
18175/18175 ━━━━━━━━━━━━━━━━━━━━ 45s 2ms/step - loss: 0.2501 - val_loss: 0.2457
Epoch 2/6
18175/18175 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - loss: 0.2443 - val_loss: 0.2440
Epoch 3/6
18175/18175 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 0.2426 - val_loss: 0.2430
Epoch 4/6
18175/18175 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 0.2416 - val_loss: 0.2423
Epoch 5/6
18175/18175 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 0.2408 - val_loss: 0.2418
Epoch 6/6
18175/18175 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 0.2402 - val_loss: 0.2415
5680/5680 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.2418
Test Loss: 0.23952385783195496


In [53]:
def predict_for_user(user_id, model, users, movies, scalerUser, scalerItem, scalerTarget):
    # Extraire les informations de l'utilisateur
    user_data = users[users['UserId'] == user_id].iloc[:, 1:].values
    # Mettre à l'échelle les données utilisateur
    user_data = scalerUser.transform(user_data)

    # Extraire les informations des films
    movie_data = movies.drop(columns=['Title', 'Year', 'runtimeMinutes', 'MovieId']).values
    # Mettre à l'échelle les données des films
    movie_data = scalerItem.transform(movie_data)

    # Répliquer les données utilisateur pour chaque film
    user_data = np.repeat(user_data, movie_data.shape[0], axis=0)

    # Faire les prédictions
    predictions = model.predict([user_data, movie_data])

    # Inverser la transformation des prédictions
    predictions = scalerTarget.inverse_transform(predictions)

    # Créer un DataFrame avec les résultats
    results = pd.DataFrame({
        'MovieId': movies['MovieId'],
        'Title': movies['Title'],
        'PredictedRating': predictions.flatten()
    })

    return results

user_id = 1
predictions = predict_for_user(user_id, model, users, movies, scalerUser, scalerItem, scalerTarget)
print(predictions)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
      MovieId                        Title  PredictedRating
0           1                    toy story         3.430732
1           2                      jumanji         3.430601
2           3             grumpier old men         3.428073
3           4            waiting to exhale         3.424573
4           5  father of the bride part ii         3.429050
...       ...                          ...              ...
3185     3948             meet the parents         3.430590
3186     3949          requiem for a dream         3.430714
3187     3950                    tigerland         3.428389
3188     3951             two family house         3.372265
3189     3952                    contender         3.427408

[3190 rows x 3 columns]


In [54]:
print('average predicted rating:', predictions['PredictedRating'].mean())
print('max predicted rating:', predictions['PredictedRating'].max())
print('min predicted rating:', predictions['PredictedRating'].min())

average predicted rating: 3.4127572
max predicted rating: 3.4307852
min predicted rating: 2.6492686
